# User Activity Monitoring and Analytics
In this presentation, we will model a simple user activity model and use logical and visual tools to analyze this data

First, let's define some basic functions to prepare environment and generate random data

In [1]:
import random
import pandas as pd
from itertools import tee
import numpy as np

_pages = """
/Login
/Login/forgotpassword
/Logout
/Home
/Home/Events
/Home/TickerPriceHistory/
/Holdings
/Holdings/Details/%w
/Activities/
/Activities/Details/%w
/Alerts
/Elections/
/Elections/Details/%w
/AE/
/AE/Dashboard
/AE/RSU
/AE/RCA
/AE/SAR
/AE/ESPP
/Help
/Contacts
/PageNotFound
/ServerError
"""

dates_range = pd.date_range(start='1/4/2018 00:00:00', end='1/4/2018 23:59:59', freq='S')

pages = _pages.split('\n')[1:-1]

def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

def rand_userId():
    return random.randrange(500000,500200)

def rand_page():
    return random.choice(pages).replace('%w',str(random.randrange(1,10000)))

def rand_timestamp():
    return dates_range[random.randrange(0,dates_range.size-1)]

def rand_vectorized_timestamp(iterations):
    pos = 0
    _generated_range = []
    for i in range(iterations):
        pos = random.randrange(pos,dates_range.size-1)
        _generated_range.append(dates_range[pos])
    return _generated_range

def generate_single_user_activity(rows):
    id = rand_userId()
    _data = []
    _timestamps = rand_vectorized_timestamp(rows)
    for element in _timestamps:
        _data.append([id, element, rand_page()])
    return pd.DataFrame(data=_data, columns=['UserId','TimeStamp','URL'])

def generate_multiple_user_activity(rows):
    _data = []
    for i in range(rows):
        _data.append([rand_userId(),rand_timestamp(), rand_page()])
    return pd.DataFrame(data=_data, columns=['userid','timestamp','page'])

def calculate_timespent():
    for index, row in data.iterrows():
        try:
            next_row = data[(data.index > index) & (data.userid == row.userid)].iloc[0]
            delta = next_row.timestamp - row.timestamp
            delta_minutes = round(delta.total_seconds()/60)
            data.at[index, 'timespent'] = delta_minutes
        except IndexError as e:
            pass
        except Exception as ex:
            print(ex)
            
def show_links(userid):
    suv = data[data.userid==userid]
    print("%s records found for user %s" % (suv.shape[0], userid))
    for (i0, row0), (i1, row1) in pairwise(suv.iterrows()):
        print("%s (%s) -> %s" % (row0.page, row0.timespent, row1.page))
        
        #data.page.value_counts().head(50).plot(kind='bar',figsize=(20,20),x='page')

## Let's generate some random data

In [4]:
data = generate_multiple_user_activity(5000)

## Now that we have data, let's look at it

In [5]:
data.head(5)

,userid,timestamp,page
0,500115,2018-01-04 17:23:41,/PageNotFound
1,500137,2018-01-04 23:13:54,/Alerts
2,500032,2018-01-04 22:45:17,/Login
3,500187,2018-01-04 01:57:33,/Home/TickerPriceHistory/
4,500055,2018-01-04 23:15:22,/AE/


## Let's sort the data by timestamp and align the indexes
We will also add an extra `timespent` column to calculate how long users have spent of specific pages

In [7]:
data = data.sort_values('timestamp').reset_index(drop=True)
data['timespent'] = np.nan
pd.to_numeric(data.timespent,errors='raise')
data.head(5)

,userid,timestamp,page,timespent
0,500116,2018-01-04 00:00:43,/AE/RSU,NaN
1,500064,2018-01-04 00:02:01,/Help,NaN
2,500180,2018-01-04 00:02:04,/ServerError,NaN
3,500175,2018-01-04 00:02:19,/Home,NaN
4,500072,2018-01-04 00:02:55,/Home/TickerPriceHistory/,NaN


## I built a function that will calculate time spent between clicks, it takes time to chain up all the records, but it can be optimized

In [8]:
%time calculate_timespent()

CPU times: user 6.68 s, sys: 10 ms, total: 6.69 s
Wall time: 6.69 s


### Let's look at the data now with time spent (in minutes) for a particular user

In [11]:
data[data.userid==500116].tail(5)

,userid,timestamp,page,timespent
2594,500116,2018-01-04 12:22:19,/AE/Dashboard,98.0
2952,500116,2018-01-04 14:00:49,/Elections/,132.0
3417,500116,2018-01-04 16:12:49,/AE/SAR,68.0
3638,500116,2018-01-04 17:20:39,/AE/RSU,170.0
4220,500116,2018-01-04 20:10:47,/AE/SAR,NaN


## Let's calculate some stats

Some hints: `data.describe()` also via `data.set_index([...]).stack.groupby(level=0).describe().unstack()`

In [12]:
print("%s unique visitors" % data.userid.unique().size)

200 unique visitors


## Time spent on pages statistics, per page

In [14]:
rows=10
metrics = dict(size="Hits",sum="Sum",max="Max",min="Min",mean="Mean")
%time data.groupby('page').timespent.agg(metrics.keys()).rename(columns=metrics).head(rows)

CPU times: user 10 ms, sys: 0 ns, total: 10 ms
Wall time: 15.9 ms


,Hits,Sum,Max,Min,Mean
page,,,,,
/AE/,228,12131.0,355.0,0.0,55.392694
/AE/Dashboard,194,12347.0,422.0,0.0,65.675532
/AE/ESPP,199,10913.0,271.0,0.0,56.838542
/AE/RCA,228,12107.0,297.0,0.0,55.792627
/AE/RSU,220,12102.0,419.0,0.0,57.904306
/AE/SAR,218,13061.0,381.0,0.0,61.032710
/Activities/,227,11095.0,256.0,0.0,50.203620
/Activities/Details/1048,1,164.0,164.0,164.0,164.000000
/Activities/Details/109,1,47.0,47.0,47.0,47.000000


## User-bases statistics

In [24]:
data.groupby(data.userid).page.value_counts()

userid  page                     
500000  /AE/RCA                      2
        /Activities/                 2
        /Alerts                      2
        /Help                        2
        /Holdings                    2
        /Home/Events                 2
        /AE/                         1
        /AE/ESPP                     1
        /AE/RSU                      1
        /Elections/                  1
        /Holdings/Details/7346       1
        /Holdings/Details/9369       1
        /Home                        1
        /Login                       1
        /Logout                      1
        /PageNotFound                1
        /ServerError                 1
500001  /Alerts                      4
        /Contacts                    3
        /Login/forgotpassword        3
        /AE/Dashboard                2
        /AE/                         1
        /AE/RCA                      1
        /AE/RSU                      1
        /Elections/Details/878